In [1]:
import os

In [2]:
%pwd

'c:\\Users\\nsika\\Videos\\TECH\\Machine Learning\\end-to-end-mlops-data-science-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\nsika\\Videos\\TECH\\Machine Learning\\end-to-end-mlops-data-science-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:

    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path

In [6]:
from machine_learning_project.constants import *
from machine_learning_project.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:

    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH
                  ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion = DataIngestionConfig(

            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion

In [8]:
import os
import zipfile
import urllib.request as request
from machine_learning_project import logger
from machine_learning_project.utils.common import get_size

In [15]:
class DataIngestion:

    def __init__(self, config : DataIngestionConfig):
        self.config = config

    
    def download_file(self):

        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(

                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f'{filename} download! with the following info \n{headers}')

        else:
            logger.info(f'File already exists of size : {get_size(Path(self.config.local_data_file))}')

    def extract_zipfile(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as f:
            f.extractall(unzip_path)

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
    
except Exception as e:
    raise e

[2023-10-23 13:41:34,183: INFO common yaml file : config\config.yaml loaded successfully]
[2023-10-23 13:41:34,195: INFO common yaml file : params.yaml loaded successfully]
[2023-10-23 13:41:34,203: INFO common Created directory at: artifacts ]
[2023-10-23 13:41:34,207: INFO common Created directory at: artifacts/data_ingestion ]


[2023-10-23 13:41:38,181: INFO 928224894 artifacts/data_ingestion/data.zip download! with the following info 
Connection: close
Content-Length: 26148
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "43594c048acfff8ebb4b5c8a05d7ed6ba7d1e876604feb1b5c9cc9abbdd42fe9"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5B12:11F78:10D1994:11897EA:653669BF
Accept-Ranges: bytes
Date: Mon, 23 Oct 2023 12:41:39 GMT
Via: 1.1 varnish
X-Served-By: cache-ams21045-AMS
X-Cache: HIT
X-Cache-Hits: 1
X-Timer: S1698064899.081687,VS0,VE1
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 687aa8bd15d179a05d680ca85ea33f338065d4ab
Expires: Mon, 23 Oct 2023 12:46:39 GMT
Source-Age: 67

]
